# RNN LSTM Sequence Analysis
This notebook takes as input an audio event detection prediction dataset. This dataset was output from a CNN model that detects the presence of the desired audio signal (4kHz tone) in a sliding window of 150ms audio samples. 

The goal of this notebook is to train an RNN model to detect the sequence of beeps emited from a washing machine appliance. This detection would then be provided to an advanced monitoring system for alerting etc.

### Input Data Description
The appliance emits a sequence of beeps, with sequence duration of one second, in the form: 

*beep-beep-beep-beep-space-space-space-space*

This pattern is repeated four times in succession.

An initial Convolution Neural Network was trained to detect the audio frequency of the beeps and its output is a series of zeros and ones for presence or absence of the target tone. Due to the large amount of redundency in the output of that network, the response was reduced by selecting every 20th prediction outcome. The reduced dataset is the input to this analysis.

### Data Preparation
The input data was further processed for use with training an RNN model. Each input value represents a timeseries sequence of audio event detections. To create the training data this input stream of ones and zeros was divided up into short sequences of values. A sliding window of size 40 values and a stride of 10 values was used.

The size of 40 was chosen as it was large enough to contain the entire target beep sequence.

This resulted in a large number of overlaping sequences that move forward in time. 

These are examples of the positive or target class:
```
1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0 : 1
0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0 : 1
0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0 : 1
0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1 : 1
```

These are examples of the negative class:
```
1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0 : 0
0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0 : 0
1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 : 0
0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 : 0
1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 : 0
0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 : 0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 : 0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 : 0
```

# Load the input files
Load the prepared input sequence data.

In [1]:
import pandas as pd
data_df = pd.read_csv('labeled_rnn_input_data.csv')
print(data_df.columns)
data_df.info()
data_df.head()

Index([u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9', u'10',
       u'11', u'12', u'13', u'14', u'15', u'16', u'17', u'18', u'19', u'20',
       u'21', u'22', u'23', u'24', u'25', u'26', u'27', u'28', u'29', u'30',
       u'31', u'32', u'33', u'34', u'35', u'36', u'37', u'38', u'39',
       u'response'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2760 entries, 0 to 2759
Data columns (total 41 columns):
0           2760 non-null int64
1           2760 non-null int64
2           2760 non-null int64
3           2760 non-null int64
4           2760 non-null int64
5           2760 non-null int64
6           2760 non-null int64
7           2760 non-null int64
8           2760 non-null int64
9           2760 non-null int64
10          2760 non-null int64
11          2760 non-null int64
12          2760 non-null int64
13          2760 non-null int64
14          2760 non-null int64
15          2760 non-null int64
16          2760 non-null int64
17      

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,response
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# The response or positive class value is stored in the last column of the dataset.
response = data_df.response
# Remove the response column from the training data.
X = data_df.drop('response', axis=1)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Test/Train Split Function for Time Series Data
For time independant data we would normally randomly select the observations from our dataset. However for time dependant data this does not make sense. This function allows you to slice the time stream at the provided cutoff point and returns the resulting train and test sets.

In [120]:
def temporal_train_test_split(X,Y, cutoff):
    X_train = X[:cutoff]
    X_test = X[cutoff:]
    Y_train = Y[:cutoff]
    Y_test = Y[cutoff:]
    return X_train, X_test, Y_train, Y_test

## Initialise Keras and Tensorflow

In [121]:
# %%time
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
# 1/3 of the data is used for validation
from sklearn.model_selection import train_test_split
X_train_df, X_test_df, y_train, y_test = temporal_train_test_split(X, response, 2100)

# fix random seed for reproducibility
seed = 1337
np.random.seed(seed)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, LSTM, TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras import backend as K

# To work-around a Keras bug with the latest TensorFlow backend the following two lines are required
import tensorflow as tf
tf.python.control_flow_ops = tf

## Model Definition

In [124]:
def make_model(input_shape):
    model = Sequential()
    model.add(LSTM(40, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(40))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim=1)) # Binary Classification
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',
                      optimizer='adadelta',
                      metrics=['binary_accuracy'])
    return model

## Training & Validation

In [125]:
sample_length=40
# fit parameters
batch_size = 64
nb_epoch = 5

# The input shape for the model needs to be defined depending on the active backend
if K.image_dim_ordering() == 'th': # Theano
    X_train = X_train_df.values.reshape(X_train_df.shape[0], 1, sample_length)
    X_test = X_test_df.values.reshape(X_test_df.shape[0], 1, sample_length)
    input_shape = (1, sample_length)
else: # TensorFlow
    X_train = X_train_df.values.reshape(X_train_df.shape[0], sample_length, 1)
    X_test = X_test_df.values.reshape(X_test_df.shape[0], sample_length, 1)
    input_shape = (sample_length, 1)

print('input_shape: ',input_shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

Y_train = np.array(y_train)
Y_test = np.array(y_test)

model = make_model(input_shape)

model.fit(np.array(X_train), Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(np.array(X_test), Y_test))

scores = model.evaluate(X_test, Y_test, verbose=0)

prob_nn = model.predict_proba(X_test, verbose=0)
auc = metrics.roc_auc_score(Y_test,prob_nn[:])
logloss= metrics.log_loss(Y_test,prob_nn[:])

('input_shape: ', (40, 1))
('X_train shape:', (2100, 40, 1))
(2100, 'train samples')
(660, 'test samples')
Train on 2100 samples, validate on 660 samples
Epoch 1/5
2100/2100 [==============================] - 10s - loss: 0.2892 - binary_accuracy: 0.9624 - val_loss: 0.0559 - val_binary_accuracy: 0.9924
Epoch 2/5
2100/2100 [==============================] - 3s - loss: 0.0372 - binary_accuracy: 0.9914 - val_loss: 0.0258 - val_binary_accuracy: 0.9924
Epoch 3/5
2100/2100 [==============================] - 3s - loss: 0.0185 - binary_accuracy: 0.9943 - val_loss: 0.0174 - val_binary_accuracy: 0.9924
Epoch 4/5
2100/2100 [==============================] - 3s - loss: 0.0131 - binary_accuracy: 0.9948 - val_loss: 0.0153 - val_binary_accuracy: 0.9939
Epoch 5/5
2100/2100 [==============================] - 3s - loss: 0.0096 - binary_accuracy: 0.9962 - val_loss: 0.0123 - val_binary_accuracy: 0.9924


In [126]:
print("%s: %.2f%%" % (model.metrics_names, scores[1]*100))
print('auc: ',auc)
print('logloss: ',logloss)

['loss', 'binary_accuracy']: 99.24%
('auc: ', 1.0)
('logloss: ', 0.01229947237615389)


### Save the Trained Model for Production Use
First train a model on ALL of the data since we have completed our model selection and evaluation step and need to save the model for new unseen data.

In [127]:
%%time
# The input shape for the model needs to be defined depending on the active backend
if K.image_dim_ordering() == 'th': # Theano
    X_all = X.values.reshape(X.shape[0], 1, sample_length)
    input_shape = (1, sample_length)
else: # TensorFlow
    X_all = X.values.reshape(X.shape[0], sample_length, 1)
    input_shape = (sample_length, 1)

CPU times: user 72 µs, sys: 7 µs, total: 79 µs
Wall time: 82 µs


In [128]:
%%time
model = make_model(input_shape)
history = model.fit(np.array(X_all), response, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1)

model.save('audio_sequence_detection.h5')

Epoch 1/5
2760/2760 [==============================] - 10s - loss: 0.1864 - binary_accuracy: 0.9707    
Epoch 2/5
2760/2760 [==============================] - 5s - loss: 0.0220 - binary_accuracy: 0.9931     
Epoch 3/5
2760/2760 [==============================] - 4s - loss: 0.0136 - binary_accuracy: 0.9935     
Epoch 4/5
2760/2760 [==============================] - 4s - loss: 0.0113 - binary_accuracy: 0.9946     
Epoch 5/5
2760/2760 [==============================] - 4s - loss: 0.0087 - binary_accuracy: 0.9971     
CPU times: user 7min 13s, sys: 40.4 s, total: 7min 54s
Wall time: 6min 57s


## Visualise some predictions
Run the predict_class operation over all the available data. Add the prediction results to the dataframe and compare with the response value where the predicted class is positive.

In [130]:
predictions = model.predict_classes(X_all)

2760/2760 [==============================] - 6s     


In [158]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
data_df['prediction'] = predictions
data_df[data_df.prediction == 1]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,response,prediction
1047,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,1,1
1660,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,1,1
1787,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1
1788,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,1,1
2000,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,1,1
2001,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,1,1
2109,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,0,1,1


Based on the above it appears that the model has perfectly learned to predict the positive class.